In [1]:
"""
VQE
"""

import numpy as np
from numpy import linalg
from qiskit.providers.aer import AerSimulator
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import X,Z,I
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info.operators import Operator

#backend = BasicAer.get_backend('statevector_simulator')
backend = AerSimulator(method='statevector')
quantum_instance = QuantumInstance(backend=backend, seed_transpiler=1)
#backend = Aer.get_backend("qasm_simulator")

#parametro per circuito di ansatz, quante volte si ripete il blocco
reps = 1

optimizer = COBYLA(tol=0.01)
intermediate_info = dict()

In [2]:
def Hamiltonian(N, J):
    """ 
    given N number of sites in the Ising Chain and J as above.
    Returns the Hamiltonian as sum of Pauli Operators
    """
    
    H = -J*(X^(I^N-2)^X) + (I^(N-1)^Z)
    for i in range (N-1):
        H += -J*((I^i)^(X^X)^(I^(N-2-i))) + ((I^i)^(Z)^(I^(N-i-1)))
    return H

def callback(nfev, parameters, energy, stddev):
    """ 
    function that saves all variables generated during the VQE
    """
    
    intermediate_info['parameters'] = parameters
    intermediate_info['nfev'] = nfev
    intermediate_info['energy'] = energy

    return None

def GS(N,J):
    """ 
    given N and J estimates parameters 
    needed by the ansatz circuit to give the Ground state 
    """
    
    H = Hamiltonian(N,J)
    ansatz = EfficientSU2(N, su2_gates=['ry'], reps=reps, entanglement='linear', insert_barriers=True)
    np.random.seed(80)
    initial_point = np.random.random(N*(reps+1))
    local_vqe = VQE(ansatz=ansatz,
                    optimizer=optimizer,
                    initial_point=initial_point,
                    quantum_instance=quantum_instance,
                    callback=callback)
    local_result = local_vqe.compute_minimum_eigenvalue(H)
    par = intermediate_info['parameters']
    
    return par, local_result.eigenvalue

In [3]:
# scegli intervallo in cui voi creare dataset e numero di punti
#Js = np.linspace(1.4, 1.5, 15)
#g = np.linspace(0.7, 0.8, 15)
#Js = np.concatenate((g, Js))
Js = np.linspace(0.25, 1.75, 20)
#name of the training set
delta = 1 

for N in range(6,7):
    f = open('dataset/data{N}_{delta}.txt','w')
    #f = open(f'DATASET_D/D_{N}.txt','w')
    for J in Js:
        
        par, E = GS(N,J)
        f.write(str(round(J,4))+', ')
        f.write(str(round(E.real,4))+', ')
        for j in range(len(par)):
            f.write(str(round(par[j],4))+', ')
        f.write('\n')

f.close()
